In [1]:
from astropy.table import Table
from astropy.io import fits
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from minisom import MiniSom
import pickle
import os

In [2]:
df_L3_info = Table.read('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_info.fits')
df_L3_info = df_L3_info.to_pandas()

In [3]:
L3_phot_data = np.loadtxt('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_averaged_phot102.txt')
phot_data = L3_phot_data[:,3::2]
phot_err_data = L3_phot_data[:,4::2]

In [4]:
data = (phot_data - np.mean(phot_data, axis=0)) / np.std(phot_data, axis=0)

In [5]:
with open("D:/SPHEREx_SOM/record/grid_search/record.pkl", "rb") as fh:
    record = pickle.load(fh)

EOFError: Ran out of input

In [13]:
def gridsearch_map(record, data, info_data):
    n = 0
    for j in record:
        n += 1
        if ("density" not in j) and ("som" in j):
            dim = j["dim"]
            som = j["som"]
            density_map = np.zeros(dim ** 2).reshape(dim, dim)
            magnitude_map = np.zeros(dim ** 2).reshape(dim, dim)
            magnitude_map.fill(np.nan)
            z_map = np.zeros(dim ** 2).reshape(dim, dim)
            z_map.fill(np.nan)
            
            labels_map_1 = som.labels_map(data, [0] * len(data))
            labels_map_2 = som.labels_map(data, tuple(map(tuple, np.concatenate((np.expand_dims(info_data["HSC_i_MAG"].values, axis = 1), np.expand_dims(info_data["z_true"].values, axis = 1)), axis = -1))))
            
            for i in labels_map_1.keys():
                density_map[int(list(i)[0]), int(list(i)[1])] = labels_map_1[i][0]
            
            print("Finish collecting data for density map.")
            
            for i in labels_map_2.keys():
                properties = np.mean(np.array(list(labels_map_2[i].keys())), axis = 0)
                magnitude_map[int(list(i)[0]), int(list(i)[1])] = properties[0]
                z_map[int(list(i)[0]), int(list(i)[1])] = properties[1]

            j["density"] = density_map
            j["magnitude"] = magnitude_map
            j["z"] = z_map

            print(f"No.{n} dim: {dim} / rate: {j['rate']} / sigma: {j['sigma']} / band_cut: {j["band_cut"]} / mag_cut: {j["mag_cut"]}")
            print(f"Topological error: {som.topographic_error(data)}")
            print(f"Quantization error: {som.quantization_error(data)}")
        
            %matplotlib inline
            plt.close()
            plt.figure(figsize = (25, 25))
            plt.subplot(221)
            plt.pcolor(som.distance_map().T, cmap='YlGn')  # plotting the distance map as background
            plt.colorbar()
            plt.title("Distance map (U-matrix)")
        
            plt.subplot(222)
            plt.pcolor(density_map.T, cmap='YlGn')
            plt.colorbar()
            plt.title("Density Map")
        
            plt.subplot(223)
            plt.pcolor(magnitude_map.T, cmap="plasma" + "_r")
            plt.colorbar()
            plt.title("HSC I Magnitude Map")
        
            plt.subplot(224)
            plt.pcolor(z_map.T, cmap="plasma")
            plt.colorbar()
            plt.title("True Redshift Map")
        
            plt.show()

            with open("D:/SPHEREx_SOM/record/grid_search/record.pkl", "wb") as fh:
                pickle.dump(record, fh)

In [ ]:
gridsearch_map(record, data, df_L3_info)

In [6]:
file_path = "D:/SPHEREx_SOM/record/grid_search/record.pkl"

# Check if the file is not empty
if os.path.getsize(file_path) > 0:
    with open(file_path, 'rb') as f:
        try:
            data = pickle.load(f)
        except EOFError:
            print('EOFError occurred while reading the file.')
else:
    print('The file is empty.')

EOFError occurred while reading the file.


In [7]:
import io
import pickle

# Use the pure-Python version, we can't see the internal state of the C version
pickle.Unpickler = pickle._Unpickler

with open("D:/SPHEREx_SOM/record/grid_search/record.pkl", 'rb') as f:
    unpickler = pickle.Unpickler(f)
    try:
        data = unpickler.load()
    except EOFError:
        pass
    print(unpickler.stack)

['_random_generator', RandomState(MT19937) at 0x23651E3AD40, '_learning_rate', 7, '_sigma', 0.1, '_input_len', 102, '_weights', array([[[ 1.88406493e+01,  2.62525596e+02, -1.79093368e+02, ...,
          8.46679473e+01, -2.92913452e+02,  2.52454334e+02],
        [ 5.65155590e+02,  5.94402348e+02,  1.30949870e+01, ...,
          3.77381324e+02, -1.02192426e+02,  8.58788989e+00],
        [-3.11433448e+02,  6.38446887e+02, -1.32931549e+02, ...,
         -2.46372898e+00, -1.53235701e+03,  1.34884342e+02],
        ...,
        [-2.09074878e+03, -1.05497931e+03, -1.74350665e+03, ...,
          6.16598210e+02,  2.09098671e+03,  3.24458058e+01],
        [-3.52839136e+02, -4.07191277e+01,  1.41052879e+02, ...,
         -1.21788042e+02, -8.19590816e+01,  2.24123702e+02],
        [-2.09853974e+02, -5.30623115e+02, -2.41630459e+02, ...,
          5.41642513e+02, -1.28119312e+03,  1.78865072e+02]],

       [[ 8.98603568e+00, -5.73844320e+01, -8.33468573e+01, ...,
         -2.27666900e+02, -1.1391309

In [9]:
a = unpickler.stack

In [10]:
a

['_random_generator',
 RandomState(MT19937) at 0x23651E3AD40,
 '_learning_rate',
 7,
 '_sigma',
 0.1,
 '_input_len',
 102,
 '_weights',
 array([[[ 1.88406493e+01,  2.62525596e+02, -1.79093368e+02, ...,
           8.46679473e+01, -2.92913452e+02,  2.52454334e+02],
         [ 5.65155590e+02,  5.94402348e+02,  1.30949870e+01, ...,
           3.77381324e+02, -1.02192426e+02,  8.58788989e+00],
         [-3.11433448e+02,  6.38446887e+02, -1.32931549e+02, ...,
          -2.46372898e+00, -1.53235701e+03,  1.34884342e+02],
         ...,
         [-2.09074878e+03, -1.05497931e+03, -1.74350665e+03, ...,
           6.16598210e+02,  2.09098671e+03,  3.24458058e+01],
         [-3.52839136e+02, -4.07191277e+01,  1.41052879e+02, ...,
          -1.21788042e+02, -8.19590816e+01,  2.24123702e+02],
         [-2.09853974e+02, -5.30623115e+02, -2.41630459e+02, ...,
           5.41642513e+02, -1.28119312e+03,  1.78865072e+02]],
 
        [[ 8.98603568e+00, -5.73844320e+01, -8.33468573e+01, ...,
          -2.

In [13]:
a[2]

'_learning_rate'